## Feature Reduction and EDA

#### 1. Select the features necessary based on the presence of the data in each column.
#### 2. The select the variables which are necessary for the analysis.

In [1]:
import sys
import time
import pyspark
import os
import shutil
import subprocess
# import sh
import re

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import compress, islice 
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.feature import CountVectorizer,  IDF, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import SparkSession
from IPython.display import clear_output
clear_output(wait = False)

warnings.filterwarnings(action='ignore')

start_time = time.time()
#Ensure we are using the right kernel
print(sys.version)
print(spark.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]
3.1.3


In [2]:
# !pip uninstall -y nltk
# !pip install nltk --upgrade --no-cache-dir
# %pip install nltk -U

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# nltk.download('popular', halt_on_error=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Tuning Spark to increase the memory

In [4]:
sc = spark.sparkContext
print('Original spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))

# Stop existing Spark environment
sc.stop()

# Waiting for the environment to stop
sleep_time = 10
print(f'Waiting for {sleep_time} seconds for the enviroment to stop...')
time.sleep(sleep_time)

# Applying new configuration and restarting Spark
conf = pyspark.SparkConf().setAll([('spark.driver.maxResultSize', '8g')])
sc = pyspark.SparkContext(conf=conf)

print('New spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))

# Starting  Spark session with configs applied
spark = SparkSession(sc).builder.getOrCreate()

Original spark.driver.maxResultSize: 1920m
Waiting for 10 seconds for the enviroment to stop...


23/03/03 21:26:35 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/03/03 21:26:35 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/03/03 21:26:35 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/03/03 21:26:35 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


New spark.driver.maxResultSize: 8g


In [5]:
# Display the spark DF in a beautified way
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

## To use legacy casting notation for date
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [6]:
path = 'gs://msca-bdp-students-bucket/shared_data/saikrishnaj/keyword_filtered_data/'
cmd = 'hadoop fs -du -s -h ' + path

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

8.9 G  8.9 G  gs://msca-bdp-students-bucket/shared_data/saikrishnaj/keyword_filtered_data



In [7]:
%time twit_filt = spark.read.parquet(path)

CPU times: user 4.46 ms, sys: 5.69 ms, total: 10.1 ms
Wall time: 8.26 s


23/03/03 21:28:59 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
%time twit_filt.count()

CPU times: user 97.5 ms, sys: 14 ms, total: 112 ms
Wall time: 43.1 s


4814661

### Filtering the features - Feature Reduction

#### Screening the data for checking the presence of data in the columns i.e. percent present for each column

In [9]:
from pyspark.sql import functions as F

twit_filt.select([(F.count(F.when(F.col(c).isNull(), c))/F.count(F.lit(1))).alias(c) for c in twit_filt.schema.names]).limit(1)


coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_in_countries,stripped_text,important
0.9996388115383409,0.0,0.9467692533285313,0.0,0.9663091544762965,1.0,0.0,0.0,0.0,0.9996388115383409,0.0,0.0,0.9494620701229017,0.9502542338910258,0.9502542338910258,0.9494620701229017,0.9494620701229017,0.0,0.0,0.9978609085873336,0.9206764505330697,0.0,0.9162109232612639,0.9160775805399384,0.9160775805399384,0.9162109232612639,0.9162109232612639,0.0,0.0,0.0,0.08786350690110893,0.10788963127414371,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
a= twit_filt.limit(10)

In [24]:
a['created_at']

0    Fri Jul 01 11:08:21 +0000 2022
1    Fri Jul 01 11:08:23 +0000 2022
2    Fri Jul 01 11:08:34 +0000 2022
3    Fri Jul 01 11:08:35 +0000 2022
4    Fri Jul 01 11:08:52 +0000 2022
5    Fri Jul 01 11:09:22 +0000 2022
6    Fri Jul 01 11:09:24 +0000 2022
7    Fri Jul 01 11:09:28 +0000 2022
8    Fri Jul 01 11:09:38 +0000 2022
9    Fri Jul 01 11:09:42 +0000 2022
Name: created_at, dtype: object

In [22]:
twit_filt.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

#### Selecting only the columns by discarding the poorly populated columns

In [50]:
user_cols = ["created_at","description","favourites_count","followers_count","friends_count","id_str",
            "name","protected","screen_name","statuses_count","verified","withheld_in_countries","location"]

tweet_cols = ["coordinates","created_at","id_str","lang","possibly_sensitive","retweeted_status",
              "tweet_text","timestamp_ms","quoted_status","text"]

retweet_cols = ["retweet_count","favorite_count","reply_count","quote_count"]

quoted_cols = ["quote_count"]

df = twit_filt.select([*[col('user.' + col_name).alias('user_' + col_name) for col_name in user_cols],
                       *[col(col_name).alias('tweet_' + col_name) for col_name in tweet_cols],
                       *[col('retweeted_status.' + col_name).alias(col_name) for col_name in retweet_cols]])\
              .withColumn('user_created_at',to_timestamp(col('user_created_at'),'EEE MMM dd HH:mm:ss zzzzz yyyy'))\
              .withColumn('tweet_created_at',to_timestamp(col('tweet_created_at'),'EEE MMM dd HH:mm:ss zzzzz yyyy'))

In [52]:
df.printSchema()

root
 |-- user_created_at: timestamp (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_favourites_count: long (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_protected: boolean (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_statuses_count: long (nullable = true)
 |-- user_verified: boolean (nullable = true)
 |-- user_withheld_in_countries: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- user_location: string (nullable = true)
 |-- tweet_coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- tweet_created_at: timestamp (nullable = true)
 |-- tweet_id_str: string (nullable = true)
 |-- tweet_lang: string (nullable = tru

In [53]:
print(f"Features before removing the columns: {len(twit_filt.columns)}")
print(f"Features after removing the columns: {len(df.columns)}")

Features before removing the columns: 41
Features after removing the columns: 27


In [54]:
print(list(set(twit_filt.columns) - set(df.columns)))

['is_quote_status', 'important', 'place', 'in_reply_to_user_id', 'source', 'retweeted_from', 'withheld_in_countries', 'quoted_status_permalink', 'entities', 'extended_entities', 'timestamp_ms', 'stripped_text', 'display_text_range', 'possibly_sensitive', 'extended_tweet', 'favorited', 'truncated', 'retweeted', 'in_reply_to_status_id_str', 'geo', 'in_reply_to_screen_name', 'quoted_status_id', 'text', 'quoted_text', 'coordinates', 'quoted_status', 'in_reply_to_user_id_str', 'in_reply_to_status_id', 'user', 'id_str', 'retweeted_status', 'lang', 'created_at', 'id', 'filter_level', 'quoted_status_id_str']


### Identifying the correct columns to select for retweet_count and reply_count

In [60]:
df_counts = df.select([
    df.tweet_retweeted_status.quoted_status.retweet_count.alias('retweeted_status_quoted_status_retweet_count'),
    df.tweet_retweeted_status.retweet_count.alias('retweeted_status_retweet_count'),
    df.tweet_quoted_status.retweet_count.alias('quoted_status_retweet_count'),
    df.retweet_count.alias("direct_retweet_count"),
    df.tweet_retweeted_status.quoted_status.reply_count.alias('retweeted_status_quoted_status_reply_count'),
    df.tweet_retweeted_status.reply_count.alias("retweeted_status_reply_count"),
    df.tweet_quoted_status.reply_count.alias("quoted_status_reply_count"),
    df.reply_count.alias("direct_reply_count")
]).limit(10000).toPandas()

In [61]:
df_counts.describe()

,retweeted_status_quoted_status_retweet_count,retweeted_status_retweet_count,quoted_status_retweet_count,direct_retweet_count,retweeted_status_quoted_status_reply_count,retweeted_status_reply_count,quoted_status_reply_count,direct_reply_count
count,654.000000,9068.000000,723.000000,9068.000000,654.000000,9068.000000,723.000000,9068.000000
mean,1117.388379,3380.666520,1229.897649,3380.666520,618.899083,616.703132,628.573997,616.703132
std,4288.029226,7446.487993,4543.353634,7446.487993,2401.409267,1741.311268,2465.052062,1741.311268
min,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,15.250000,16.000000,14.000000,16.000000,3.000000,2.000000,2.000000,2.000000
50%,164.500000,256.500000,157.000000,256.500000,45.000000,22.000000,45.000000,22.000000
75%,534.250000,2568.250000,562.500000,2568.250000,399.500000,268.000000,425.000000,268.000000
max,88945.000000,100953.000000,88945.000000,100953.000000,31706.000000,27406.000000,31706.000000,27406.000000


In [82]:
#selecting retweeted_status.retweet_count for EDA (retweet and reply counts are pretty much the same)

In [81]:
#Getting the count of original tweets with no retweets at all.
original_tweets = df.filter("tweet_retweeted_status.retweet_count is null")
original_tweets.count()

519452

### Identiying the correct columns to select for the location information

In [67]:
df_loc = df.select([
    df.user_withheld_in_countries,
    df.user_location.alias('user_location'),
    df.tweet_coordinates.alias("direct_coordinates"),
    df.tweet_retweeted_status.coordinates.alias("retweeted_status_coordinates"),
    df.tweet_retweeted_status.geo.coordinates.alias("retweeted_status_geo_coordinates"),
    df.tweet_retweeted_status.place.bounding_box.coordinates.alias("retweeted_status_place_box_coordinates"),
    df.tweet_retweeted_status.place.country.alias("retweeted_status_place_country"),
    df.tweet_retweeted_status.place.country_code.alias("retweeted_status_place_country_code"),
    df.tweet_retweeted_status.place['name'].alias("retweeted_status_place_name"),
    df.tweet_retweeted_status.place.full_name.alias("retweeted_status_place_full_name"),
    df.tweet_quoted_status.coordinates.alias("quoted_coordinates"),
    
]).limit(10000).toPandas()

In [68]:
df_loc.describe()

,user_withheld_in_countries,user_location,direct_coordinates,retweeted_status_coordinates,retweeted_status_geo_coordinates,retweeted_status_place_box_coordinates,retweeted_status_place_country,retweeted_status_place_country_code,retweeted_status_place_name,retweeted_status_place_full_name,quoted_coordinates
count,10000,6276,4,0,0,442,442,442,442,442,0
unique,1,3775,2,0,0,90,16,16,90,90,0
top,[],United States,"([78.57329071, 13.36094689], Point)",NaN,NaN,"[[[3.198942, 6.482676], [3.198942, 6.698546], [3.43719, 6.698546], [3.43719, 6.482676]]]",Nigeria,NG,Ikeja,"Ikeja, Nigeria",NaN
freq,10000,141,3,NaN,NaN,286,293,293,286,286,NaN


In [69]:
#selecting user_withheld_in_countries as a filter (already done in the previous notebook) and using direct_coordnates and user_location for EDA
df.filter("user_withheld_in_countries is not null").count()

4814661

### Preparing the final DataFrame for the EDA and analyses by selecting the relevant columns

In [64]:
df.printSchema()

root
 |-- user_created_at: timestamp (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_favourites_count: long (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_protected: boolean (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_statuses_count: long (nullable = true)
 |-- user_verified: boolean (nullable = true)
 |-- user_withheld_in_countries: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- user_location: string (nullable = true)
 |-- tweet_coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- tweet_created_at: timestamp (nullable = true)
 |-- tweet_id_str: string (nullable = true)
 |-- tweet_lang: string (nullable = tru

In [89]:
df_final_eda = df.select([
    df.user_created_at,
    df.user_id_str.alias("user_id"),
    df.user_name,
    df.tweet_retweeted_status.user['name'].alias("original_tweeter_username"),
    df.user_description,
    df.user_followers_count.alias("followers_count"),
    df.user_verified.alias("is_user_verified"),
    df.user_location,
    df.tweet_id_str.alias("tweet_id"),
    df.tweet_created_at,
    df.tweet_retweeted_status.retweet_count.alias("retweet_count"),
    df.tweet_retweeted_status.reply_count.alias("reply_count"),
    df.tweet_retweeted_status.alias("retweeted_status"),
    df.tweet_tweet_text.alias("tweet_text"),
    df.tweet_text.alias("text") ])
    #.filter("retweet_count is not null")

In [90]:
df_final_eda.limit(10)

user_created_at,user_id,user_name,original_tweeter_username,user_description,followers_count,is_user_verified,user_location,tweet_id,tweet_created_at,retweet_count,reply_count,retweeted_status,tweet_text,text
2021-02-05 19:40:14,1357775619669524487,Rahel yoseif,ቲቲ ጓል 70 🇵🇱እንደር...,null,1006,false,null,1607724278773948416,2022-12-27 13:05:12,20,0,"{null, Tue Dec 27...",On 21 September 2...,rt @titieyi: on 2...
2022-12-07 05:14:24,1600357706119802880,sumang gujer,Ramchandra Yadav,null,12,false,null,1607724293760364546,2022-12-27 13:05:16,20,0,"{null, Tue Dec 27...",Out of 461 govt c...,rt @rcyadav51: ou...
2012-09-08 11:36:14,810751759,ann prosser #FBPE...,BladeoftheSun,null,1635,false,null,1607724313003847681,2022-12-27 13:05:20,62,2,"{null, Tue Dec 27...",You don’t have to...,rt @bladeofthes: ...
2020-11-01 11:31:05,1322863687699472384,Cami🪧,BladeoftheSun,"Designer, Tech ad...",9642,false,GPS determined,1607724327947960323,2022-12-27 13:05:24,63,2,"{null, Tue Dec 27...",You don’t have to...,rt @bladeofthes: ...
2022-08-02 17:52:07,1554525318274174976,Madhuri Soni,null,null,46,false,null,1607724334076026880,2022-12-27 13:05:25,null,null,null,Girl students are...,girl students are...
2022-01-02 15:59:51,1477670767399620609,Sharvan Lal Bhati,Ramchandra Yadav,Great thought,1308,false,"Bikaner, India",1607724364912562179,2022-12-27 13:05:33,21,0,"{null, Tue Dec 27...",Out of 461 govt c...,rt @rcyadav51: ou...
2014-12-25 10:01:17,2940581980,Katy Veronica 💙💛,BladeoftheSun,🏴󠁧󠁢󠁷󠁬󠁳󠁿fro...,90,false,"Somerset, England",1607724381916069888,2022-12-27 13:05:37,64,2,"{null, Tue Dec 27...",You don’t have to...,rt @bladeofthes: ...
2021-10-16 14:35:57,1449383144386220036,KIND HEART,All India Radio News,KINDNESS IN HUMAN...,319,false,null,1607724382021095425,2022-12-27 13:05:37,6,1,"{null, Tue Dec 27...",National Educatio...,rt @airnewsalerts...
2022-12-12 14:43:08,1602312969379282944,Mahipal,null,null,25,false,null,1607724414791208961,2022-12-27 13:05:45,null,null,null,54 agriculture co...,54 agriculture co...
2022-04-27 06:22:22,1519200229765283842,Chandan Gurjar,null,राज्य सरकार की जन...,131,false,Jaipur Amer,1607724465139650561,2022-12-27 13:05:57,null,null,null,With focus on gir...,with focus on gir...


## Save the filtered and feature selected data

In [91]:
df_final_eda.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/saikrishnaj/final_eda')

In [92]:
df_final_eda.count()

4814661